In [1]:
MIN_NUMBER_OF_QUBITS = 1
MAX_NUMBER_OF_QUBITS = 5

In [2]:
# Ansatz Circuit Construction
import jaqalpaq as jql
from qscout.v1.std.jaqal_gates import ALL_GATES

def _prepare_state(
    circuit_parameters,
    number_of_qubits,
    circuit_builder,
):
    """Constructs the circuit to prepare the initial state that is used during measurement of all cliques"""
    assert len(circuit_parameters) == number_of_qubits
    assert number_of_qubits > 0
    
    
    # Must create three distinct lists of parameters (theta, theta/2, -theta) for re-evaluation of the circuit 
    #     using different theta values when batching
    full_circuit_parameters = [
        circuit_builder.let("theta{}".format(i), circuit_parameter)
        for i, circuit_parameter in enumerate(circuit_parameters)
    ]
    positive_halved_parameters = [
        circuit_builder.let("(1/2)theta{}".format(i), circuit_parameter / 2)
        for i, circuit_parameter in enumerate(circuit_parameters)
    ]
    negative_halved_parameters = [
        circuit_builder.let("-(1/2)theta{}".format(i), -circuit_parameter / 2)
        for i, circuit_parameter in enumerate(circuit_parameters)
    ]

    # The reduced-unarily-encoded EGO circuit
    qubits = circuit_builder.register("q", number_of_qubits)
    circuit_builder.gate("prepare_all")

    #     Ry
    circuit_builder.gate("Ry", qubits[0], full_circuit_parameters[0])

    for target_qubit_index in range(1, number_of_qubits):
        #######     Controlled Ry         #######
        target_qubit = qubits[target_qubit_index]
        control_qubit = qubits[target_qubit_index - 1]
        positive_halved_parameter = positive_halved_parameters[target_qubit_index]
        negative_halved_parameter = negative_halved_parameters[target_qubit_index]

        circuit_builder.gate("Ry", target_qubit, positive_halved_parameter)
        circuit_builder = _add_cnot_gate(circuit_builder, control_qubit, target_qubit)
        circuit_builder.gate("Ry", target_qubit, negative_halved_parameter)
        circuit_builder = _add_cnot_gate(circuit_builder, control_qubit, target_qubit)

    return circuit_builder, qubits


def _add_cnot_gate(circuit_builder, control, target):
    """CNOT Gate from Maslov (2017)"""
    circuit_builder.gate("Sy", control)
    circuit_builder.gate("Sxx", control, target)
    circuit_builder.gate("Sxd", control)
    circuit_builder.gate("Sxd", target)
    circuit_builder.gate("Syd", control)
    return circuit_builder


def _add_hadamard_gate(circuit_builder, qubit):
    """Hadamard Gate from JAQAL (https://www.sandia.gov/quantum/Projects/quantum_assembly_spec.pdf)"""
    circuit_builder.gate("Sy", qubit)
    circuit_builder.gate("Px", qubit)
    return circuit_builder

In [3]:
def create_clique1_circuit(circuit_parameters, number_of_qubits):
    """Constructs the ansatz circuit used to measure clique 1"""
    builder = jql.core.circuitbuilder.CircuitBuilder(native_gates=ALL_GATES)

    builder, _ = _prepare_state(
        circuit_parameters,
        number_of_qubits,
        builder,
    )
    builder.gate("measure_all")
    return builder.build()


def create_clique2_circuit(circuit_parameters, number_of_qubits):
    """Constructs the ansatz circuit used to measure clique 2"""
    builder = jql.core.circuitbuilder.CircuitBuilder(native_gates=ALL_GATES)
    
    builder, qubits = _prepare_state(
        circuit_parameters,
        number_of_qubits,
        builder,
    )
    
    # Hadamard rotation on all qubits to change measurement context
    for i in range(number_of_qubits):
        builder = _add_hadamard_gate(builder, qubits[i])
    builder.gate("measure_all")
    return builder.build()


def create_clique3_circuit(circuit_parameters, number_of_qubits):
    """Constructs the ansatz circuit used to measure clique 3"""
    assert number_of_qubits > 1
    builder = jql.core.circuitbuilder.CircuitBuilder(native_gates=ALL_GATES)

    builder, qubits = _prepare_state(
        circuit_parameters,
        number_of_qubits,
        builder,
    )

    builder = _add_hadamard_gate(builder, qubits[1])
    builder = _add_cnot_gate(builder, qubits[0], qubits[1])
    builder = _add_hadamard_gate(builder, qubits[0])

    builder.gate("measure_all")
    return builder.build()


def create_clique4_circuit(circuit_parameters, number_of_qubits):
    """Constructs the ansatz circuit used to measure clique 4"""
    assert number_of_qubits > 2
    builder = jql.core.circuitbuilder.CircuitBuilder(native_gates=ALL_GATES)
    
    builder, qubits = _prepare_state(
        circuit_parameters,
        number_of_qubits,
        builder,
    )

    builder = _add_hadamard_gate(builder, qubits[2])
    builder = _add_cnot_gate(builder, qubits[1], qubits[2])
    builder = _add_hadamard_gate(builder, qubits[1])

    builder.gate("measure_all")
    return builder.build()

In [4]:
from jaqalpaq.generator import generate_jaqal_program
from qscout.v1.std.ionsim import IonSimErrorModel
jql.core.result.ProbabilisticSubcircuit.CUTOFF_FAIL = 1e-4
jql.core.result.ProbabilisticSubcircuit.CUTOFF_WARN = 1e-4
import numpy as np


def get_bitstrings_from_measured_probability_distribution(
    number_of_qubits, number_of_samples, probability_distribution
):
    bitstrings = [
        format(i, "0{}b".format(number_of_qubits))[::-1]
        for i in range(2 ** number_of_qubits)
    ]
    
#     counts = [
#         int(probability * number_of_samples) for probability in probability_distribution
#     ]
    
    measurements = np.random.choice(bitstrings, size=number_of_samples, replace=True, p=probability_distribution)
#     [bitstrings[i] for i, count in enumerate(counts) for _ in range(count)]
    
#     import pdb; pdb.set_trace()
    return measurements.tolist()


def get_batched_probabilities_for_clique_on_simulator(
    circuit_generator,
    circuit_parameters_list,
    number_of_qubits,
):
    backend = IonSimErrorModel(
        number_of_qubits,
        model="standard",
        params=["dpower12", "dfreq1", "dphase1", "dtime"],
        v0={"dpower12": 5e-4, "dfreq1": 5e3, "dphase1": 5e-2, "dtime": 5e-3},
        sigmas={"dpower12": 5e-4, "dfreq1": 5e3, "dphase1": 5e-2, "dtime": 5e-3},
    )
    all_probabilities = []
    for circuit_parameters in circuit_parameters_list:
        circuit = circuit_generator(circuit_parameters, number_of_qubits)
        sim_result = jql.emulator.run_jaqal_circuit(circuit, backend=backend)
        all_probabilities.append(list(sim_result.subcircuits[0].probability_by_int))
    return all_probabilities


def get_batched_probabilities_for_clique_on_hardware(
    circuit_generator,
    circuit_parameters_list,
    number_of_qubits,
    number_of_samples
):
    # Initialize ansatz circuit using placeholder (theta=0) for parameter values
    empty_parameters = [0 for _ in circuit_parameters_list[0]]
    circuit = circuit_generator(empty_parameters, number_of_qubits)
    
    # Construct jaqal code for circuit
    circuit_code = "\n".join(
        ["from UserPulseDefinitions.LaserRamseyGates usepulses *"]
        + [generate_jaqal_program(circuit)]
    )

    # Construct dictionary to use when batching circuits
    batch_dictionary = {"__repeats__": number_of_samples}
    
    # Adding parameter values to loop over in parallel when batching 
    #    For example, when given theta1 = [pi/4, pi/2, pi]
    #                            (1/2)theta1 = [pi/8, pi/4, pi/2]
    #                            -(1/2)theta1 = [-pi/8, -pi/4, -pi/2]
    #                            theta2 = [pi/6, pi/8, pi/10]
    #                            (1/2)theta2 = [pi/12, pi/16, pi/20]
    #                            -(1/2)theta2 = [-pi/12, -pi/16, -pi/20]
    #        Then we should run three circuits, with parameter values:
    #            [theta1, (1/2)theta1, -(1/2)theta1, theta2, (1/2)theta2, -(1/2)theta2] = 
    #            [pi/4, pi/8, -pi/8, pi/6, pi/12, -pi/12]
    #            [pi/2, pi/4, -pi/4, pi/8, pi/16, -pi/16]
    #            [pi, pi/2, -pi/2, pi/10, pi/20, -pi/20]
    #    len(circuit_parameters_list) number of circuits should be run)
    batch_dictionary.update(
        {
            "theta{}".format(i): [
                parameters[i] for parameters in circuit_parameters_list
            ]
            for i, _ in enumerate(circuit_parameters_list[0])
        }
    )
    batch_dictionary.update(
        {
            "(1/2)theta{}".format(i).format(i): [
                parameters[i] / 2 for parameters in circuit_parameters_list
            ]
            for i, _ in enumerate(circuit_parameters_list[0])
        }
    )
    batch_dictionary.update(
        {
            "-(1/2)theta{}".format(i).format(i): [
                -parameters[i] / 2 for parameters in circuit_parameters_list
            ]
            for i, _ in enumerate(circuit_parameters_list[0])
        }
    )

    print("Executing a batch with {} circuits".format(len(circuit_parameters_list)))
    # Run code on device and record probabilities of measurement outcomes indexed by parameter values
    return [list(experiment.subcircuits[0].probability_by_int)
            for experiment in jql.run.run_jaqal_batch(circuit_code, batch_dictionary)]


In [5]:
# Circuit Parameter Values for Evaluations of Point/Error Model
M1_POINT_PARAMETER_VALUES = [3.85532]
M2_POINT_PARAMETER_VALUES = [3.28449, 3.73368]
M3_POINT_PARAMETER_VALUES = [3.11251, 3.3066, 3.72502]
M4_POINT_PARAMETER_VALUES = [3.14757, 3.09809, 3.31778, 3.72279]
M5_POINT_PARAMETER_VALUES = [3.14036, 3.15256, 3.09041, 3.3243, 3.72197]

POINT_PARAMETER_VALUES_BY_NUMBER_OF_QUBITS = [
    M1_POINT_PARAMETER_VALUES,
    M2_POINT_PARAMETER_VALUES,
    M3_POINT_PARAMETER_VALUES,
    M4_POINT_PARAMETER_VALUES,
    M5_POINT_PARAMETER_VALUES
]

In [6]:
# Run Point Error Model Experiment for 1, 2, 3, 4, and 5 qubits
import json
import os

import datetime

date = datetime.datetime.now()

device = "simulator"
RESULTS_DIRECTORY = "lmg-results/point/{}_{}-{}-{}_naive/".format(device, date.month, date.day, date.year)
if not os.path.exists(RESULTS_DIRECTORY):
    os.makedirs(RESULTS_DIRECTORY)
    
NUMBER_OF_SAMPLES = 10000

for number_of_qubits, circuit_parameters in zip(
    range(MIN_NUMBER_OF_QUBITS, MAX_NUMBER_OF_QUBITS+1),
    POINT_PARAMETER_VALUES_BY_NUMBER_OF_QUBITS,
):
    print(circuit_parameters)
    # "0" chooses the ground-state to prepare, you can change this to "1" or "2" 
    #     to choose 1st or 2nd excited state respectively
    updated_circuit_parameters = []
    for parameter in circuit_parameters:
        # Fit parameters to -pi -> pi range
        if parameter > np.pi:
            parameter -= 2*np.pi
            print(parameter)
        updated_circuit_parameters.append(parameter)
    circuit_parameters = updated_circuit_parameters
        
    cliques_to_measure = [create_clique1_circuit, create_clique2_circuit]
    if number_of_qubits > 1:
        cliques_to_measure.append(create_clique3_circuit)
    if number_of_qubits > 2:
        cliques_to_measure.append(create_clique4_circuit)

#     # Will return a 2D list indexed first by clique measurements, then each list item being the 
#     #    measured probability distribution
#     all_probability_measurements = [
#         get_batched_probabilities_for_clique_on_hardware(
#                 clique,
#                 [circuit_parameters],
#                 number_of_qubits,
#                 NUMBER_OF_SAMPLES,
#         )[0] for clique in cliques_to_measure
#     ]
    
    # Will return a 2D list indexed first by clique measurements, then each list item being the 
    #    measured probability distribution
    all_probability_measurements = [
        get_batched_probabilities_for_clique_on_simulator(
                clique,
                [circuit_parameters],
                number_of_qubits,
        )[0] for clique in cliques_to_measure
    ]
    
    data_filename = RESULTS_DIRECTORY + "probability_data_{}_qubits.json".format(number_of_qubits)
    with open(data_filename, "w") as f:
        f.write(json.dumps(all_probability_measurements))
    f.close()
    
    all_measurements = [
        get_bitstrings_from_measured_probability_distribution(
            number_of_qubits, NUMBER_OF_SAMPLES, probability_distribution
        )
        for probability_distribution in all_probability_measurements
    ]

    measurement_data_filename = RESULTS_DIRECTORY + "measurement_data_{}_qubits.json".format(number_of_qubits)
    
    with open(measurement_data_filename, "w") as f:
        f.write(json.dumps(all_measurements))
    f.close()


[3.85532]
-2.4278653071795864


FileNotFoundError: [Errno 2] No such file or directory: '/Users/williamsimon/Desktop/Research/QSCOUT/ionsim-source-code/qscout-gatemodels-ionsim/src/qscout/v1/std/ionsim/standard/R_phi_theta_dpower12_dfreq1_dphase1_dtime.pyg'

In [7]:
# # Circuit Parameter Values for Evaluations of Grid Model
# M1_GRID_PARAMETERS = [
#     [-2.42787],
#     [-2.16607],
#     [-1.90427],
#     [-1.64247],
#     [-1.38067],
#     [-1.11887],
#     [-0.857072],
#     [-0.595273],
#     [-0.333473],
#     [-0.0716738],
#     [0.190126],
#     [0.451925],
#     [0.713724],
#     [0.975524],
#     [1.23732],
#     [1.49912],
#     [1.76092],
#     [2.02272],
#     [2.28452],
#     [2.54632],
#     [2.80812],
#     [3.06992],
#     [-2.9514653071795864],
#     [-2.6896653071795864],
#     [-2.4278653071795864],
# ]

# M2_GRID_PARAMETERS = [
#     [0.9020536235925247, 0.4629547279403564],
#     [0.9020536235925247, 0.4629547279403564],
#     [0.9020536235925247, 0.4629547279403564],
#     [0.9020536235925247, 0.4629547279403564],
#     [0.9020536235925247, 0.4629547279403564],
#     [0.7171600358014212, 1.4239980535976393],
#     [0.9087836254301958, -1.5593470255289201],
#     [1.5707963267948966, 3.141592653589793],
#     [2.153336811107822, 1.9668989490500308],
#     [2.45541797209311, 0.8092031073519976],
#     [2.2652945924214527, 0.9752324854302117],
#     [2.3820444460328476, -0.7412620791480595],
#     [2.9412578112666736, -2.2605713275803963],
#     [-2.678897673620007, 2.5886608359047507],
#     [-2.2652945924214523, 0.975232485430212],
#     [-2.45541797209311, 0.8092031073519976],
#     [-2.374386577184911, -0.36249758790453157],
#     [-1.9390642202315362, -1.5119388208478153],
#     [-1.2919004335677968, -2.766845692323793],
#     [-0.7171600358014212, 1.4239980535976398],
#     [-0.9020536235925247, 0.46295472794035586],
#     [-0.9020536235925247, 0.46295472794035586],
#     [-0.9020536235925247, 0.46295472794035586],
#     [-0.9020536235925247, 0.46295472794035586],
#     [-0.9020536235925247, 0.46295472794035586],
# ]

# M3_GRID_PARAMETERS = [
#     [1.0990922242409917, 0.7520908006923863, 0.33377948918979117],
#     [1.0990922242409917, 0.7520908006923863, 0.33377948918979117],
#     [1.0990922242409917, 0.7520908006923863, 0.33377948918979117],
#     [1.0990922242409917, 0.7520908006923863, 0.33377948918979117],
#     [1.0990922242409917, 0.7520908006923863, 0.33377948918979117],
#     [1.6053234798853584, -3.0676072953435276, 1.11271410522402],
#     [1.721219341511268, 2.672060734325206, 2.513407948267981],
#     [1.9029984273469487, 1.889090694174488, -2.4944678499533675],
#     [2.0428557391964723, 1.2168889994992955, -0.8296006506949629],
#     [2.0684497060044125, 1.1822378064807582, 1.5461767716124681],
#     [2.881204471449564, -2.029366589936621, 1.2858981713231774],
#     [3.00294030977484, -2.66316973934001, 2.8661435479593487],
#     [-3.0778556742670364, 2.76080976998911, -2.1059337367750217],
#     [-2.9124277752688394, 2.07775540368426, -0.5827924318871442],
#     [-2.881204471449564, 2.0293665899366213, 1.2858981713231774],
#     [-2.0684497060044125, -1.1822378064807575, 1.5461767716124684],
#     [-1.9672416539630575, -1.8209015706801601, -3.0208812759240566],
#     [-1.7899862646819145, -2.597168169051291, -1.7653169420038228],
#     [-1.6355716782758192, 3.106590105303592, -0.41810124245001745],
#     [-1.6053234798853584, 3.0676072953435276, 1.1127141052240193],
#     [-1.0990922242409917, -0.7520908006923861, 0.3337794891897907],
#     [-1.0990922242409917, -0.7520908006923861, 0.3337794891897907],
#     [-1.0990922242409917, -0.7520908006923861, 0.3337794891897907],
#     [-1.0990922242409917, -0.7520908006923861, 0.3337794891897907],
#     [-1.0990922242409917, -0.7520908006923861, 0.3337794891897907],
# ]

In [8]:
# # Run Grid Model Experiment for 1, 2, and 3 qubits
# import json
    
# NUMBER_OF_SAMPLES = 2 ** 13

# for number_of_qubits, grid_parameters in zip(
#     [1, 2, 3],
#     [M1_GRID_PARAMETERS, M2_GRID_PARAMETERS, M3_GRID_PARAMETERS],
# ):
#     RESULTS_DIRECTORY = "results/grid/hardware/{}/".format(number_of_qubits)
#     if not os.path.exists(RESULTS_DIRECTORY):
#         os.makedirs(RESULTS_DIRECTORY)
    
#     cliques_to_measure = [create_clique1_circuit, create_clique2_circuit]
#     if number_of_qubits > 1:
#         cliques_to_measure.append(create_clique3_circuit)
#     if number_of_qubits > 2:
#         cliques_to_measure.append(create_clique4_circuit)

#     # Will return a 3D list, first index being clique measurements, second index being parameter values, then each
#     #    list item being the measured probability distribution
#     all_probability_measurements = [
#         get_batched_measurements_for_clique_on_hardware(
#                 clique,
#                 grid_parameters,
#                 number_of_qubits,
#                 NUMBER_OF_SAMPLES,
#         ) for clique in cliques_to_measure
#     ]
    
#     data_filename = RESULTS_DIRECTORY + "probability_data_{}_qubits.json".format(number_of_qubits)
#     with open(data_filename, "w") as f:
#         f.write(json.dumps(all_probability_measurements))
#     f.close()
    
#     # Reorder probability distributions
#     reordered_probability_measurements = [
#         [[] for _ in all_probability_measurements]
#         for _ in all_probability_measurements[0]
#     ]
#     for i, clique_probability_distributions in enumerate(all_probability_measurements):
#         for j, probability_distribution in enumerate(clique_probability_distributions):
#             reordered_probability_measurements[j][i] = probability_distribution

#     for i, probability_distributions_for_parameters in enumerate(reordered_probability_measurements):
#         measurements_for_parameters = []
#         for j, probability_distribution_for_clique in enumerate(probability_distributions_for_parameters):
#             measurements = get_bitstrings_from_measured_probability_distribution(number_of_qubits, NUMBER_OF_SAMPLES, probability_distribution_for_clique)
#             measurements_for_parameters.append(measurements)

#         measurement_data_filename = (
#             RESULTS_DIRECTORY
#             + "measurement_data_point_{}.json".format(i+1)
#         )
#         with open(measurement_data_filename, "w") as f:
#             f.write(json.dumps(measurements_for_parameters))
#         f.close()
